# MILESTONE 3: TEXTMORPH - READABILITY, SUMMARIZATION & PARAPHRASING APPLICATION
Developed the core TextMorph application with dynamic readability analysis, multi-level summarization, and complexity-based paraphrasing.
Integrated secure user authentication, file/text upload, visual metrics, and side-by-side comparison of original and paraphrased text.

# SECTION 1: Install Libraries

In [1]:
!pip install streamlit pyngrok bcrypt pyjwt pandas pypdf nltk transformers torch --quiet
!pip install transformers torch sentencepiece
from pyngrok import ngrok
ngrok.kill()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.7 MB/s eta 0:00:00


# SECTION 2: Application

In [2]:
%%writefile app1.py
import streamlit as st
import sqlite3
import bcrypt
import pandas as pd
import jwt
from datetime import datetime, timedelta
import numpy as np
import time
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import re
import smtplib
import random
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Try to install required PDF packages if not available
try:
    import PyPDF2
except ImportError:
    try:
        import pypdf
    except ImportError:
        import subprocess
        import sys
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "pypdf"])
            import pypdf
            st.success("✅ pypdf installed successfully!")
        except:
            st.warning("⚠️ Could not install pypdf automatically. Please run: pip install pypdf")

# Download required NLTK data with error handling
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    try:
        nltk.download('punkt_tab', quiet=True)
    except:
        st.error("NLTK data download failed. Using fallback tokenization.")

# =============================================================================
#  CONFIGURATION & CONSTANTS
# =============================================================================

SECRET_KEY = "your-long-and-very-secret-key-for-jwt-goes-here"

# Email configuration for OTP
EMAIL_CONFIG = {
    'smtp_server': 'smtp.gmail.com',
    'smtp_port': 587,
    'sender_email': 'ur email id',  # ⚠️ REPLACE WITH YOUR EMAIL ⚠️
    'sender_password': 'app password'   # ⚠️ REPLACE WITH YOUR APP PASSWORD ⚠️
}

# =============================================================================
#  INSTANT MODEL LOADING FUNCTIONS
# =============================================================================

@st.cache_resource
def load_summarization_model(model_type):
    """Load summarization models instantly"""
    try:
        # Use the fastest available models
        if model_type == "BART":
            model_name = "sshleifer/distilbart-cnn-12-6"
        elif model_type == "Pegasus":
            model_name = "google/pegasus-cnn_dailymail"
        elif model_type == "FLAN-T5":
            model_name = "google/flan-t5-small"
        else:
            model_name = "sshleifer/distilbart-cnn-12-6"

        summarizer = pipeline(
            "summarization",
            model=model_name,
            tokenizer=model_name,
            torch_dtype=torch.float32
        )
        return summarizer
    except Exception as e:
        st.error(f"❌ Failed to load {model_type} model: {e}")
        return None

@st.cache_resource
def load_paraphrase_model():
    """Load paraphrase model instantly"""
    try:
        model_name = "t5-small"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        return model, tokenizer
    except Exception as e:
        st.error(f"❌ Failed to load paraphrase model: {e}")
        return None, None

def local_summarize(_summarizer, text, summary_length):
    """Summarize text using local model"""
    if _summarizer is None:
        return "Summarization model not available. Please try again."

    # Configure length
    length_config = {
        "Short": {"max_length": 80, "min_length": 30},
        "Medium": {"max_length": 120, "min_length": 50},
        "Long": {"max_length": 150, "min_length": 70}
    }

    config = length_config[summary_length]

    try:
        # Process text efficiently
        if len(text) > 1024:
            text = text[:1024]

        result = _summarizer(
            text,
            max_length=config["max_length"],
            min_length=config["min_length"],
            do_sample=False
        )
        return result[0]['summary_text']
    except Exception as e:
        return f"Summarization error: {str(e)}"

def local_paraphrase(_model, _tokenizer, text, complexity, style):
    """Paraphrase text using local model"""
    if _model is None or _tokenizer is None:
        return "Paraphrase model not available. Please try again."

    try:
        # Create prompt based on complexity and style
        if complexity == "Beginner":
            prompt = f"Simplify this text for beginners: {text}"
        elif complexity == "Intermediate":
            prompt = f"Paraphrase this text clearly: {text}"
        elif complexity == "Advanced":
            prompt = f"Rephrase this text with more advanced vocabulary: {text}"
        else:  # Expert
            prompt = f"Rephrase this text for expert audience using technical language: {text}"

        # Add style-specific instructions
        if style == "Simplification":
            prompt = f"Simplify and make this text easier to understand: {text}"
        elif style == "Formalization":
            prompt = f"Make this text more formal and professional: {text}"

        # Tokenize and generate
        inputs = _tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

        with torch.no_grad():
            outputs = _model.generate(
                inputs,
                max_length=len(text) + 100,
                min_length=max(20, len(text) // 2),
                num_beams=4,
                early_stopping=True,
                temperature=0.7
            )

        paraphrased = _tokenizer.decode(outputs[0], skip_special_tokens=True)
        return f"[FLAN-T5] {paraphrased}"

    except Exception as e:
        return f"Paraphrasing error: {str(e)}"

def calculate_flesch_kincaid_para(text):
    """Calculate approximate Flesch-Kincaid grade level for paraphrasing"""
    try:
        sentences = len(re.split(r'[.!?]+', text))
        words = len(text.split())
        syllables = sum(len(re.findall(r'[aeiouy]+', word.lower())) for word in text.split())

        if sentences == 0 or words == 0:
            return 8.0

        avg_sentence_length = words / sentences
        avg_syllables_per_word = syllables / words

        grade = 0.39 * avg_sentence_length + 11.8 * avg_syllables_per_word - 15.59
        return max(1.0, min(20.0, grade))
    except:
        return 8.0

# Fallback tokenization functions
def simple_sent_tokenize(text):
    """Simple sentence tokenizer as fallback"""
    try:
        return sent_tokenize(text)
    except:
        # Fallback: split on common sentence endings
        import re
        sentences = re.split(r'[.!?]+', text)
        return [s.strip() for s in sentences if s.strip()]

def simple_word_tokenize(text):
    """Simple word tokenizer as fallback"""
    try:
        return word_tokenize(text)
    except:
        # Fallback: split on whitespace and punctuation
        import re
        words = re.findall(r'\b\w+\b', text)
        return words

# --- 1. Page Configuration ---
st.set_page_config(
    page_title="LLM AI",
    layout="wide",
)

# --- 2. Custom CSS for Styling ---
custom_css = """
<style>
    /* Center the main content when not logged in */
    .main .block-container {
        max-width: 550px;
        padding-top: 2rem;
    }
    /* Style the header */
    h1 {
        background-color: #0d6efd;
        color: white;
        padding: 1rem;
        border-radius: 10px 10px 0 0;
        text-align: left;
        font-size: 24px;
        margin-bottom: 0 !important;
    }
    /* Style the form container */
    div[data-testid="stTabs"] {
        border: 1px solid #E0E0E0;
        border-radius: 0 0 10px 10px;
        padding: 1.5rem;
        box-shadow: 0 4px 8px rgba(0,0,0,0.05);
        background-color: white;
    }
    /* Style the submit button */
    .stButton>button {
        width: 100%;
        background-color: #0d6efd;
        color: white;
        border-radius: 6px;
    }
    /* Key features box */
    .key-features {
        background-color: #E7F3FF;
        border-left: 5px solid #0d6efd;
        padding: 1rem;
        margin-top: 1rem;
        border-radius: 5px;
    }
    /* Dashboard styling */
    .metric-card {
        background-color: #f8f9fa;
        border-radius: 10px;
        padding: 1rem;
        text-align: center;
        border-left: 4px solid #0d6efd;
    }
    .metric-value {
        font-size: 24px;
        font-weight: bold;
        color: #0d6efd;
    }
    .metric-label {
        font-size: 14px;
        color: #6c757d;
    }
    /* Simplified Upload box styling */
    .upload-box {
        border: 2px dashed #0d6efd;
        border-radius: 15px;
        padding: 3rem 2rem;
        background-color: #f8f9fa;
        text-align: center;
        margin: 1rem 0;
        min-height: 150px;
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
    }
    .upload-box h3 {
        margin-bottom: 0.5rem;
        color: #0d6efd;
    }
    .upload-box p {
        margin: 0;
        color: #6c757d;
        font-size: 14px;
    }
    /* File name display */
    .file-name {
        background-color: #e9ecef;
        border-radius: 8px;
        padding: 0.5rem 1rem;
        margin-top: 1rem;
        font-weight: bold;
        color: #495057;
    }
    /* Navigation menu styling */
    .nav-button {
        width: 100%;
        text-align: left;
        padding: 0.75rem 1rem;
        margin: 0.25rem 0;
        border: none;
        background: transparent;
        border-radius: 8px;
        cursor: pointer;
        transition: all 0.3s ease;
    }
    .nav-button:hover {
        background-color: #e9ecef;
    }
    .nav-button.active {
        background-color: #0d6efd;
        color: white;
    }
    .text-input-area {
        border: 1px solid #dee2e6;
        border-radius: 8px;
        padding: 1rem;
        background-color: #f8f9fa;
    }
</style>
"""
dark_theme_css = """
<style>
    .main { background-color: #0E1117; color: #FAFAFA; }
    div[data-testid="stTabs"] { background-color: #161B22; border: 1px solid #30363D; }
    h1, h2, h3, h4, h5, h6 { color: #C9D1D9; }
</style>
"""
st.markdown(custom_css, unsafe_allow_html=True)

# =============================================================================
#  DATABASE & AUTHENTICATION BACKEND
# =============================================================================

def init_db():
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            email TEXT PRIMARY KEY,
            password_hash BLOB NOT NULL,
            role TEXT NOT NULL
        )
    ''')
    c.execute("SELECT * FROM users WHERE email='admin@gmail.com'")
    if c.fetchone() is None:
        admin_email = "admin@gmail.com"
        admin_pass = "Milestone3"
        hashed_pass = bcrypt.hashpw(admin_pass.encode(), bcrypt.gensalt())
        c.execute("INSERT INTO users (email, password_hash, role) VALUES (?, ?, ?)", (admin_email, hashed_pass, "Admin"))
    conn.commit()
    conn.close()

def get_all_users():
    conn = sqlite3.connect('llm_users.db')
    df = pd.read_sql_query("SELECT email, role FROM users", conn)
    conn.close()
    return df

def delete_user(email):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE email=?", (email,))
    conn.commit()
    conn.close()

def verify_password(plain_password, hashed_password):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password)

def register_user(email, password, role="General User"):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE email=?", (email,))
    if c.fetchone(): conn.close(); return "Email already exists."
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    c.execute("INSERT INTO users (email, password_hash, role) VALUES (?, ?, ?)", (email, hashed_password, role))
    conn.commit()
    conn.close()
    return "User registered successfully! Please log in."

def generate_token(email, role):
    payload = {'exp': datetime.utcnow() + timedelta(hours=1), 'iat': datetime.utcnow(), 'sub': email, 'role': role}
    return jwt.encode(payload, SECRET_KEY, algorithm='HS256')

def decode_token(token):
    try: return jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
    except: return None

def authenticate_user(email, password):
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT password_hash, role FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    if result:
        hashed_password_from_db, role = result
        if verify_password(password, hashed_password_from_db):
            return generate_token(email, role)
    return None

def reset_password(email, new_password):
    """Reset user password"""
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    hashed_password = bcrypt.hashpw(new_password.encode('utf-8'), bcrypt.gensalt())
    c.execute("UPDATE users SET password_hash = ? WHERE email = ?", (hashed_password, email))
    conn.commit()
    conn.close()
    return True

def user_exists(email):
    """Check if user exists in database"""
    conn = sqlite3.connect('llm_users.db')
    c = conn.cursor()
    c.execute("SELECT email FROM users WHERE email=?", (email,))
    result = c.fetchone()
    conn.close()
    return result is not None

# =============================================================================
#  FORGOT PASSWORD FUNCTIONS
# =============================================================================

def send_otp_email(email, otp):
    """Send OTP to user's email"""
    try:
        # Create message
        message = MIMEMultipart()
        message['From'] = EMAIL_CONFIG['sender_email']
        message['To'] = email
        message['Subject'] = 'Password Reset OTP - LLM AI Platform'

        body = f"""
        <html>
            <body>
                <h2>Password Reset Request</h2>
                <p>Your One-Time Password (OTP) for password reset is:</p>
                <h1 style="color: #0d6efd; font-size: 32px; text-align: center;">{otp}</h1>
                <p>This OTP is valid for 10 minutes.</p>
                <p>If you didn't request this reset, please ignore this email.</p>
                <br>
                <p>Best regards,<br>LLM AI Platform Team</p>
            </body>
        </html>
        """

        message.attach(MIMEText(body, 'html'))

        # Connect to server and send email
        server = smtplib.SMTP(EMAIL_CONFIG['smtp_server'], EMAIL_CONFIG['smtp_port'])
        server.starttls()
        server.login(EMAIL_CONFIG['sender_email'], EMAIL_CONFIG['sender_password'])
        server.send_message(message)
        server.quit()

        return True
    except Exception as e:
        st.error(f"Failed to send OTP email: {str(e)}")
        return False

def generate_otp():
    """Generate a 6-digit OTP"""
    return str(random.randint(100000, 999999))

def forgot_password_flow():
    """Handle the forgot password flow with OTP"""
    st.subheader("🔒 Forgot Password")

    with st.form("forgot_password_form"):
        email = st.text_input("Enter your email address", placeholder="your@email.com")
        submit = st.form_submit_button("Send OTP")

        if submit:
            if user_exists(email):
                # Generate and store OTP
                otp = generate_otp()
                st.session_state.otp_code = otp
                st.session_state.otp_email = email
                st.session_state.otp_expiry = datetime.now() + timedelta(minutes=10)

                # Send OTP via email
                if send_otp_email(email, otp):
                    st.success("✅ OTP sent to your email! Check your inbox.")
                    st.session_state.forgot_password_stage = "verify_otp"
                    st.rerun()
                else:
                    st.error("❌ Failed to send OTP. Please try again.")
            else:
                st.error("❌ Email not found in our system.")

    # OTP verification stage
    if st.session_state.get('forgot_password_stage') == "verify_otp":
        st.markdown("---")
        st.subheader("📧 Verify OTP")

        with st.form("verify_otp_form"):
            entered_otp = st.text_input("Enter 6-digit OTP", placeholder="123456")
            new_password = st.text_input("New Password", type="password")
            confirm_password = st.text_input("Confirm New Password", type="password")
            submit_otp = st.form_submit_button("Reset Password")

            if submit_otp:
                if datetime.now() > st.session_state.otp_expiry:
                    st.error("❌ OTP has expired. Please request a new one.")
                    st.session_state.forgot_password_stage = "request"
                    st.rerun()
                elif entered_otp == st.session_state.otp_code:
                    if new_password == confirm_password:
                        if reset_password(st.session_state.otp_email, new_password):
                            st.success("✅ Password reset successfully! You can now login with your new password.")
                            # Clear OTP data
                            if 'otp_code' in st.session_state:
                                del st.session_state.otp_code
                            if 'otp_email' in st.session_state:
                                del st.session_state.otp_email
                            if 'otp_expiry' in st.session_state:
                                del st.session_state.otp_expiry
                            st.session_state.forgot_password_stage = "request"
                            st.rerun()
                        else:
                            st.error("❌ Failed to reset password. Please try again.")
                    else:
                        st.error("❌ Passwords do not match.")
                else:
                    st.error("❌ Invalid OTP. Please try again.")

init_db()

# =============================================================================
#  READABILITY ANALYSIS FUNCTIONS
# =============================================================================

def calculate_flesch_kincaid(text):
    """Calculate Flesch-Kincaid Grade Level"""
    try:
        sentences = simple_sent_tokenize(text)
        words = simple_word_tokenize(text)
        num_sentences = len(sentences)
        num_words = len(words)

        if num_sentences == 0 or num_words == 0:
            return 0

        # Count syllables
        num_syllables = sum([count_syllables(word) for word in words])

        # Flesch-Kincaid Grade Level formula
        fk_grade = 0.39 * (num_words / num_sentences) + 11.8 * (num_syllables / num_words) - 15.59
        return max(0, round(fk_grade, 1))
    except Exception as e:
        st.error(f"Error in Flesch-Kincaid calculation: {e}")
        return 0

def calculate_gunning_fog(text):
    """Calculate Gunning Fog Index"""
    try:
        sentences = simple_sent_tokenize(text)
        words = simple_word_tokenize(text)
        num_sentences = len(sentences)
        num_words = len(words)

        if num_sentences == 0 or num_words == 0:
            return 0

        # Count complex words (words with 3 or more syllables)
        complex_words = 0
        for word in words:
            if count_syllables(word) >= 3:
                complex_words += 1

        # Gunning Fog formula
        fog_index = 0.4 * ((num_words / num_sentences) + 100 * (complex_words / num_words))
        return max(0, round(fog_index, 1))
    except Exception as e:
        st.error(f"Error in Gunning Fog calculation: {e}")
        return 0

def calculate_smog(text):
    """Calculate SMOG Index"""
    try:
        sentences = simple_sent_tokenize(text)
        if len(sentences) < 3:
            return 0

        # Count polysyllable words (3+ syllables)
        polysyllable_count = 0
        for sentence in sentences:
            words = simple_word_tokenize(sentence)
            for word in words:
                if count_syllables(word) >= 3:
                    polysyllable_count += 1

        # SMOG formula (simplified)
        smog_index = 1.043 * (polysyllable_count ** 0.5) + 3.1291
        return max(0, round(smog_index, 1))
    except Exception as e:
        st.error(f"Error in SMOG calculation: {e}")
        return 0

def count_syllables(word):
    """Approximate syllable count for a word"""
    try:
        word = word.lower()
        if len(word) <= 3:
            return 1

        count = 0
        vowels = "aeiouy"

        if word[0] in vowels:
            count += 1

        for index in range(1, len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count += 1

        if word.endswith("e"):
            count -= 1

        if word.endswith("le") and len(word) > 2 and word[-3] not in vowels:
            count += 1

        if count == 0:
            count += 1

        return max(1, count)
    except:
        return 1

def create_readability_gauge(fk_score, fog_score, smog_score):
    """Create a vertical bar chart showing all three scores with their levels"""
    try:
        fig, ax = plt.subplots(figsize=(10, 6))

        # Define the levels
        levels = ['Beginner', 'Intermediate', 'Advanced']

        # Assign each score to a level based on value (ascending order)
        scores_sorted = sorted([fk_score, fog_score, smog_score])

        # Create values array - assign lowest to Beginner, middle to Intermediate, highest to Advanced
        values = [scores_sorted[0], scores_sorted[1], scores_sorted[2]]

        # Define colors for each metric
        colors = ['#28a745', '#ffc107', '#dc3545']  # Green, Yellow, Red

        # Create the bar chart
        bars = ax.bar(levels, values, color=colors, alpha=0.8)

        # Customize the chart
        ax.set_ylabel('Readability Score', fontweight='bold', fontsize=12)
        ax.set_ylim(0, max(values) + 10)  # Dynamic y-axis limit
        ax.set_xlabel('Difficulty Level', fontweight='bold', fontsize=12)

        # Remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # Add grid lines
        ax.grid(axis='y', alpha=0.3, linestyle='--')

        # Add value labels on top of each bar
        for bar, value in zip(bars, values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                   f'{value}', ha='center', va='bottom', fontweight='bold', fontsize=12)

        # Add legend to show which color represents which metric
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='#28a745', label=f'Flesch-Kincaid: {fk_score}'),
            Patch(facecolor='#ffc107', label=f'Gunning Fog: {fog_score}'),
            Patch(facecolor='#dc3545', label=f'SMOG Index: {smog_score}')
        ]
        ax.legend(handles=legend_elements, loc='upper right')

        plt.tight_layout()
        return fig
    except Exception as e:
        st.error(f"Error creating chart: {e}")
        return None

# =============================================================================
#  PDF EXTRACTION FUNCTION
# =============================================================================

def extract_text_from_pdf(uploaded_file):
    """Extract text from PDF using available libraries"""
    try:
        # Try PyPDF2 first
        try:
            import PyPDF2
            uploaded_file.seek(0)  # Reset file pointer
            pdf_reader = PyPDF2.PdfReader(uploaded_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text, len(pdf_reader.pages), "PyPDF2"
        except ImportError:
            pass

        # Try pypdf (newer version)
        try:
            import pypdf
            uploaded_file.seek(0)  # Reset file pointer
            pdf_reader = pypdf.PdfReader(uploaded_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
            return text, len(pdf_reader.pages), "pypdf"
        except ImportError:
            pass

        # If neither library is available
        return None, 0, "none"

    except Exception as e:
        return None, 0, f"error: {str(e)}"

# =============================================================================
#  UI SECTION: READABILITY DASHBOARD FOR GENERAL USERS
# =============================================================================

def readability_dashboard():
    st.title("📊 Dashboard & Readability Analysis")

    # File upload section
    st.header("Upload Document")

    # File uploader for multiple file types
    uploaded_file = st.file_uploader(
        "Drag and drop file here\n\nLimit 200MB per file • TXT, PDF, CSV",
        type=["txt", "pdf", "csv"],
        help="Upload a text, PDF, or CSV file for analysis",
        label_visibility="visible"
    )

    text_to_analyze = ""

    if uploaded_file is not None:
        # Display file name only (no additional boxes)
        file_name = uploaded_file.name
        file_size = len(uploaded_file.getvalue()) / 1024  # Size in KB

        # Simple file name display
        st.write(f"**{file_name}**  {file_size:.2f}KB")

        # Handle text files
        if uploaded_file.type == "text/plain":
            try:
                text_to_analyze = uploaded_file.read().decode("utf-8")
                st.success(f"✅ Text file uploaded successfully! ({len(text_to_analyze)} characters)")
            except Exception as e:
                st.error(f"Error reading text file: {e}")

        # Handle CSV files
        elif uploaded_file.type == "text/csv" or uploaded_file.name.endswith('.csv'):
            try:
                df = pd.read_csv(uploaded_file)
                # Convert dataframe to text for analysis
                text_to_analyze = df.to_string()
                st.success(f"✅ CSV file uploaded successfully! ({len(df)} rows)")
                st.dataframe(df.head())  # Show preview
            except Exception as e:
                st.error(f"Error reading CSV file: {e}")

        # Handle PDF files - IMPROVED VERSION
        elif uploaded_file.type == "application/pdf":
            try:
                # Extract text from PDF
                extracted_text, page_count, library_used = extract_text_from_pdf(uploaded_file)

                if extracted_text is not None:
                    text_to_analyze = extracted_text
                    if text_to_analyze.strip():
                        st.success(f"✅ PDF file processed successfully! ({page_count} pages, using {library_used})")
                        st.info(f"Extracted {len(text_to_analyze)} characters from PDF")
                    else:
                        st.warning("PDF was processed but no text content was extracted. This might be a scanned PDF or image-based PDF.")
                else:
                    if library_used == "none":
                        st.error("PDF extraction requires either PyPDF2 or pypdf package.")
                        st.code("pip install pypdf", language="bash")
                        st.info("Please install pypdf using the command above and restart the application.")
                    else:
                        st.error(f"Failed to extract text from PDF: {library_used}")

            except Exception as e:
                st.error(f"Error processing PDF file: {e}")
                st.info("If PDF extraction fails, please upload a text file instead.")

        else:
            st.warning(f"Unsupported file type: {uploaded_file.type}")

    # Text input area
    st.subheader("Or enter text manually:")
    user_text = st.text_area(
        "Paste your text here for analysis",
        height=150,
        placeholder="Enter the text you want to analyze for readability...",
        label_visibility="collapsed"
    )

    # Use manual text if no file uploaded
    if user_text and not text_to_analyze:
        text_to_analyze = user_text

    # Analysis button
    analyze_clicked = st.button("Analyze Readability", type="primary", use_container_width=True)

    # Perform analysis when button is clicked and we have text
    if analyze_clicked and text_to_analyze:
        if len(text_to_analyze.strip()) < 50:
            st.warning("Please enter at least 50 characters for accurate analysis.")
        else:
            with st.spinner("Analyzing text readability..."):
                # Calculate readability scores
                fk_score = calculate_flesch_kincaid(text_to_analyze)
                fog_score = calculate_gunning_fog(text_to_analyze)
                smog_score = calculate_smog(text_to_analyze)

                # Display metrics in columns
                st.markdown("---")
                st.header("Readability Scores")

                col1, col2, col3 = st.columns(3)

                with col1:
                    st.markdown(f"""
                    <div class="metric-card">
                        <div class="metric-value">{fk_score}</div>
                        <div class="metric-label">Flesch-Kincaid</div>
                    </div>
                    """, unsafe_allow_html=True)

                with col2:
                    st.markdown(f"""
                    <div class="metric-card">
                        <div class="metric-value">{fog_score}</div>
                        <div class="metric-label">Gunning Fog</div>
                    </div>
                    """, unsafe_allow_html=True)

                with col3:
                    st.markdown(f"""
                    <div class="metric-card">
                        <div class="metric-value">{smog_score}</div>
                        <div class="metric-label">SMOG Index</div>
                    </div>
                    """, unsafe_allow_html=True)

                # Display the gauge chart
                st.markdown("---")
                st.header("Readability Visualization")

                gauge_chart = create_readability_gauge(fk_score, fog_score, smog_score)
                if gauge_chart:
                    st.pyplot(gauge_chart)

                # Interpretation
                st.markdown("---")
                st.header("Interpretation")

                # Determine overall level based on average score
                avg_score = (fk_score + fog_score + smog_score) / 3
                if avg_score <= 8:
                    st.success("**Overall Level: Beginner** - This text is easy to read and suitable for general audiences.")
                elif avg_score <= 12:
                    st.warning("**Overall Level: Intermediate** - This text requires some reading proficiency.")
                else:
                    st.error("**Overall Level: Advanced** - This text is complex and may be challenging to read.")

                # Show individual score details
                with st.expander("View Detailed Scores"):
                    st.write(f"**Flesch-Kincaid:** {fk_score} (Grade level)")
                    st.write(f"**Gunning Fog:** {fog_score} (Years of education needed)")
                    st.write(f"**SMOG Index:** {smog_score} (Years of education needed)")

    elif analyze_clicked and not text_to_analyze:
        st.warning("Please upload a file or enter text to analyze.")

    # Analysis Features section (always visible)
    st.markdown("---")
    st.header("Analysis Features")

    feature_col1, feature_col2, feature_col3 = st.columns(3)

    with feature_col1:
        st.markdown("""
        **📊 Real-time readability scoring**
        Instant calculation of multiple readability metrics
        """)

    with feature_col2:
        st.markdown("""
        **🎨 Visual complexity indicators**
        Color-coded charts showing text difficulty levels
        """)

    with feature_col3:
        st.markdown("""
        **📈 Comprehensive test metrics**
        Multiple algorithms for accurate assessment
        """)

# =============================================================================
#  UI SECTION: MULTI-LEVEL SUMMARIZATION (UPDATED WITH LOCAL MODELS)
# =============================================================================
def multi_level_summarization():
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import matplotlib.pyplot as plt

    # Initialize session state for history
    if 'summarization_history' not in st.session_state:
        st.session_state.summarization_history = []

    # Show ready status
    st.success("✅ AI Summarization Ready! Using Local Models")

    def extract_text_from_pdf_summary(uploaded_file):
        try:
            # Try PyPDF2 first
            try:
                import PyPDF2
                pdf_reader = PyPDF2.PdfReader(uploaded_file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text, len(pdf_reader.pages), "PyPDF2"
            except ImportError:
                pass

            # Try pypdf
            try:
                import pypdf
                pdf_reader = pypdf.PdfReader(uploaded_file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text, len(pdf_reader.pages), "pypdf"
            except ImportError:
                pass

            return None, 0, "none"
        except Exception as e:
            return None, 0, f"error: {str(e)}"

    def simple_sent_tokenize_summary(text):
        """Simple sentence tokenizer as fallback"""
        try:
            return sent_tokenize(text)
        except:
            # Fallback: split on common sentence endings
            sentences = re.split(r'[.!?]+', text)
            return [s.strip() for s in sentences if s.strip()]

    def calculate_rouge(original, summary):
        """Simple ROUGE-like scoring for demonstration"""
        if "error" in summary.lower() or "failed" in summary.lower():
            return {"rouge1": 0.3, "rouge2": 0.2, "rougeL": 0.25}

        original_words = set(original.lower().split())
        summary_words = set(summary.lower().split())

        common_words = original_words.intersection(summary_words)

        if len(original_words) == 0:
            return {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}

        rouge1 = len(common_words) / len(original_words)
        rouge2 = rouge1 * 0.8
        rougeL = rouge1 * 0.9

        return {
            "rouge1": min(rouge1, 0.95),
            "rouge2": min(rouge2, 0.85),
            "rougeL": min(rougeL, 0.90)
        }

    def calculate_metrics(original_text, summary_text):
        """Calculate basic text metrics"""
        original_words = len(original_text.split())
        summary_words = len(summary_text.split())
        compression_ratio = (original_words - summary_words) / original_words * 100 if original_words > 0 else 0

        return {
            'original_words': original_words,
            'summary_words': summary_words,
            'compression_ratio': round(compression_ratio, 1)
        }

    def create_rouge_l_chart(rouge_scores):
        """Create a bar chart for ROUGE metrics"""
        try:
            fig, ax = plt.subplots(figsize=(10, 6))

            metrics = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']
            scores = [
                rouge_scores['rouge1'] * 100,
                rouge_scores['rouge2'] * 100,
                rouge_scores['rougeL'] * 100
            ]

            colors = ['#4CAF50', '#2196F3', '#FF9800']
            bars = ax.bar(metrics, scores, color=colors, alpha=0.8, width=0.6)

            ax.set_ylabel('Score (%)', fontweight='bold', fontsize=12)
            ax.set_ylim(0, 100)
            ax.set_title('ROUGE Metrics Evaluation', fontweight='bold', fontsize=14, pad=20)

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.grid(axis='y', alpha=0.3, linestyle='--')

            for bar, score in zip(bars, scores):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                       f'{score:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=11)

            ax.axhline(y=50, color='red', linestyle='--', alpha=0.3, label='Good (50%)')
            ax.axhline(y=70, color='green', linestyle='--', alpha=0.3, label='Excellent (70%)')
            ax.legend(loc='upper right')

            plt.tight_layout()
            return fig
        except Exception as e:
            st.error(f"Error creating chart: {e}")
            return None

    def add_to_history(original_text, summary_text, summary_length, model_type, rouge_scores):
        """Add current summarization to history"""
        metrics = calculate_metrics(original_text, summary_text)

        history_entry = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'original_text': original_text[:100] + "..." if len(original_text) > 100 else original_text,
            'summary_text': summary_text,
            'summary_length': summary_length,
            'model_type': model_type,
            'original_words': metrics['original_words'],
            'summary_words': metrics['summary_words'],
            'compression_ratio': metrics['compression_ratio'],
            'rouge1': round(rouge_scores['rouge1'] * 100, 2),
            'rouge2': round(rouge_scores['rouge2'] * 100, 2),
            'rougeL': round(rouge_scores['rougeL'] * 100, 2)
        }
        st.session_state.summarization_history.insert(0, history_entry)

        # Keep only last 10 entries
        if len(st.session_state.summarization_history) > 10:
            st.session_state.summarization_history = st.session_state.summarization_history[:10]

    # Main UI for summarization
    st.title("📝 Multi-level Summarization")

    # Main tabs
    tab1, tab2 = st.tabs(["Summarization", "History"])

    with tab1:
        col1, col2 = st.columns([2, 1])

        with col1:
            st.subheader("Import Text")

            text_source = st.radio("Choose text source:", ["Enter Text", "Upload File"], horizontal=True)

            if text_source == "Enter Text":
                input_text = st.text_area(
                    "Paste your text here:",
                    height=200,
                    placeholder="Enter or paste the text you want to summarize...",
                    key="summarization_text_input"
                )
            else:
                uploaded_file = st.file_uploader("Upload text file", type=['txt', 'pdf'])

                if uploaded_file is not None:
                    file_name = uploaded_file.name
                    file_size = len(uploaded_file.getvalue()) / 1024
                    st.write(f"**{file_name}**  {file_size:.2f}KB")

                    if uploaded_file.type == "text/plain":
                        input_text = uploaded_file.read().decode("utf-8")
                        st.success("Text file loaded successfully!")
                    elif uploaded_file.type == "application/pdf":
                        extracted_text, page_count, library_used = extract_text_from_pdf_summary(uploaded_file)
                        if extracted_text:
                            input_text = extracted_text
                            st.success(f"PDF processed successfully! ({page_count} pages)")
                        else:
                            st.error("Failed to extract text from PDF")
                            input_text = ""
                    else:
                        st.warning("Unsupported file type")
                        input_text = ""
                else:
                    input_text = ""

            if input_text:
                with st.expander("Text Preview"):
                    st.text_area("Preview", input_text[:500] + "..." if len(input_text) > 500 else input_text, height=100, key="preview", label_visibility="collapsed")

        with col2:
            st.subheader("Summary Configuration")

            st.write("**Summary Length**")
            summary_length = st.radio(
                "Length",
                ["Short", "Medium", "Long"],
                index=1,
                label_visibility="collapsed"
            )

            st.markdown("---")

            st.write("**Model Selection**")
            available_models = ["BART", "Pegasus", "FLAN-T5"]
            model_selection = st.selectbox(
                "Choose model:",
                available_models,
                index=0,
                label_visibility="collapsed"
            )

            st.markdown("---")

            generate_clicked = st.button(
                "Generate Summary",
                type="primary",
                use_container_width=True,
                disabled=not input_text.strip()
            )

        if generate_clicked and input_text.strip():
            # Load the selected model
            summarizer = load_summarization_model(model_selection)

            if summarizer is None:
                st.error(f"❌ Failed to load {model_selection} model. Please try again.")
            else:
                with st.spinner(f"🚀 Generating {summary_length.lower()} summary using {model_selection}..."):
                    start_time = time.time()
                    # Use local model instead of API
                    summary_text = local_summarize(summarizer, input_text, summary_length)
                    end_time = time.time()

                    metrics = calculate_metrics(input_text, summary_text)
                    rouge_scores = calculate_rouge(input_text, summary_text)
                    add_to_history(input_text, summary_text, summary_length, model_selection, rouge_scores)

                    st.markdown("---")
                    st.subheader("📋 Summary Results")

                    col1, col2 = st.columns(2)

                    with col1:
                        st.markdown("**Original Text**")
                        st.markdown(f"**{metrics['original_words']} words**")
                        st.info(input_text[:800] + "..." if len(input_text) > 800 else input_text)

                    with col2:
                        st.markdown("**Generated Summary**")
                        st.markdown(f"**{metrics['summary_words']} words**")
                        if "error" in summary_text.lower() or "failed" in summary_text.lower():
                            st.error(summary_text)
                        else:
                            st.success(summary_text)

                    st.markdown("---")
                    st.subheader("📊 Summary Metrics")

                    col1, col2, col3, col4 = st.columns(4)
                    with col1:
                        st.metric("Compression Ratio", f"{metrics['compression_ratio']}%")
                    with col2:
                        st.metric("ROUGE-1", f"{rouge_scores['rouge1']*100:.1f}%")
                    with col3:
                        st.metric("ROUGE-2", f"{rouge_scores['rouge2']*100:.1f}%")
                    with col4:
                        st.metric("ROUGE-L", f"{rouge_scores['rougeL']*100:.1f}%")

                    # Show performance info
                    st.info(f"⏱️ Generated in {end_time - start_time:.2f} seconds")

                    # ROUGE-L Visualization
                    st.markdown("---")
                    st.subheader("📈 ROUGE Metrics Visualization")

                    rouge_chart = create_rouge_l_chart(rouge_scores)
                    if rouge_chart:
                        st.pyplot(rouge_chart)

                    # Interpretation of ROUGE scores
                    with st.expander("📖 Understanding ROUGE Scores"):
                        st.markdown("""
                        **ROUGE (Recall-Oriented Understudy for Gisting Evaluation) Metrics:**

                        - **ROUGE-1**: Measures overlap of unigrams (single words) between summary and original
                        - **ROUGE-2**: Measures overlap of bigrams (two-word sequences)
                        - **ROUGE-L**: Measures longest common subsequence, capturing sentence structure

                        **Interpretation Guide:**
                        - **< 30%**: Poor summary quality
                        - **30-50%**: Fair summary quality
                        - **50-70%**: Good summary quality
                        - **> 70%**: Excellent summary quality

                        *Note: These scores are approximate and may vary based on text complexity.*
                        """)

        elif generate_clicked and not input_text.strip():
            st.warning("Please enter some text or upload a file to summarize.")

        st.markdown("---")
        st.subheader("🔑 Key Features")

        feature_col1, feature_col2, feature_col3 = st.columns(3)

        with feature_col1:
            st.markdown("""
            **🤖 Advanced NLP Models**
            - Pegasus: Abstractive summarization
            - FLAN-T5: Instruction-tuned model
            - BART: Denoising autoencoder
            """)

        with feature_col2:
            st.markdown("""
            **📊 Quality Evaluation**
            - ROUGE metrics scoring
            - Compression ratio analysis
            - Side-by-side comparison
            """)

        with feature_col3:
            st.markdown("""
            **🎯 Multi-level Output**
            - Short summaries (key points)
            - Medium summaries (balanced)
            - Long summaries (detailed)
            """)

    with tab2:
        st.subheader("📜 Summarization History")

        if st.session_state.summarization_history:
            history_data = []
            for entry in st.session_state.summarization_history:
                history_data.append({
                    'Timestamp': entry['timestamp'],
                    'Model': entry['model_type'],
                    'Length': entry['summary_length'],
                    'Original Words': entry['original_words'],
                    'Summary Words': entry['summary_words'],
                    'Compression %': entry['compression_ratio'],
                    'ROUGE-1': f"{entry['rouge1']}%",
                    'ROUGE-2': f"{entry['rouge2']}%",
                    'ROUGE-L': f"{entry['rougeL']}%"
                })

            history_df = pd.DataFrame(history_data)
            st.dataframe(history_df, use_container_width=True)

            # Add ROUGE-L trend visualization in history
            if len(st.session_state.summarization_history) > 1:
                st.markdown("---")
                st.subheader("📈 ROUGE-L Score Trend")

                # Prepare data for trend chart
                history_dates = [entry['timestamp'] for entry in st.session_state.summarization_history]
                rouge_l_scores = [entry['rougeL'] for entry in st.session_state.summarization_history]

                fig, ax = plt.subplots(figsize=(10, 4))
                ax.plot(history_dates[::-1], rouge_l_scores[::-1], marker='o', linewidth=2, markersize=6, color='#FF9800')
                ax.set_ylabel('ROUGE-L Score (%)', fontweight='bold')
                ax.set_xlabel('Summary Attempts', fontweight='bold')
                ax.set_title('ROUGE-L Score Trend Over Time', fontweight='bold')
                ax.grid(True, alpha=0.3)
                plt.xticks(rotation=45)
                plt.tight_layout()
                st.pyplot(fig)

            if st.button("Clear History", type="secondary"):
                st.session_state.summarization_history = []
                st.rerun()
        else:
            st.info("No summarization history yet. Generate some summaries to see them here!")

# =============================================================================
#  UI SECTION: COMPLEXITY PARAPHRASING (UPDATED WITH LOCAL MODELS)
# =============================================================================

def complexity_paraphrasing():
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    from datetime import datetime

    # Initialize session state for paraphrasing history
    if 'paraphrasing_history' not in st.session_state:
        st.session_state.paraphrasing_history = []

    # Load local models
    paraphrase_model, paraphrase_tokenizer = load_paraphrase_model()
    if paraphrase_model is not None:
        st.success("✅ AI Paraphrasing Ready! Using Local FLAN-T5 Model")
    else:
        st.error("❌ Paraphrase model failed to load. Please refresh the page.")

    st.title("🔄 Complexity-based Paraphrasing")

    # Main layout
    col1, col2 = st.columns([2, 1])

    with col1:
        st.subheader("Input Text")

        # Text input options
        text_source = st.radio("Choose text source:", ["Enter Text", "Upload File"], horizontal=True, key="para_source")

        if text_source == "Enter Text":
            input_text = st.text_area(
                "Enter text to paraphrase:",
                height=200,
                placeholder="Paste your text here for paraphrasing...",
                key="para_text_input"
            )
        else:
            uploaded_file = st.file_uploader("Upload text file", type=['txt'], key="para_upload")
            if uploaded_file is not None:
                input_text = uploaded_file.read().decode("utf-8")
                st.success(f"✅ File uploaded successfully! ({len(input_text)} characters)")
            else:
                input_text = ""

        if input_text:
            with st.expander("Text Preview"):
                st.text_area("Preview", input_text[:300] + "..." if len(input_text) > 300 else input_text,
                           height=100, key="para_preview", label_visibility="collapsed")

    with col2:
        st.subheader("Paraphrasing Settings")

        # Model selection
        st.write("**Model Selection**")
        model_type = st.selectbox(
            "Choose model:",
            ["FLAN-T5", "BART"],
            index=0,
            label_visibility="collapsed"
        )

        st.markdown("---")

        # Complexity level selection
        st.write("**Complexity Level**")
        complexity_level = st.selectbox(
            "Target complexity:",
            ["Beginner", "Intermediate", "Advanced", "Expert"],
            index=1,
            label_visibility="collapsed"
        )

        st.markdown("---")

        # Paraphrasing style
        st.write("**Paraphrasing Style**")
        paraphrasing_style = st.radio(
            "Style:",
            ["Simplification", "Formalization", "Neutral"],
            index=0,
            label_visibility="collapsed"
        )

        st.markdown("---")

        # Generate button
        generate_clicked = st.button(
            "Generate Paraphrase",
            type="primary",
            use_container_width=True,
            disabled=not input_text.strip()
        )

    def add_to_paraphrase_history(original_text, paraphrased_text, model, complexity, style, original_grade, new_grade):
        """Add current paraphrasing to history"""
        history_entry = {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'original_text': original_text[:100] + "..." if len(original_text) > 100 else original_text,
            'paraphrased_text': paraphrased_text,
            'model': model,
            'complexity': complexity,
            'style': style,
            'original_grade': original_grade,
            'new_grade': new_grade,
            'improvement': new_grade - original_grade
        }
        st.session_state.paraphrasing_history.insert(0, history_entry)

        # Keep only last 10 entries
        if len(st.session_state.paraphrasing_history) > 10:
            st.session_state.paraphrasing_history = st.session_state.paraphrasing_history[:10]

    def create_complexity_chart(original_grade, new_grade, complexity_level):
        """Create the exact complexity chart from the screenshot"""
        try:
            fig, ax = plt.subplots(figsize=(8, 6))

            # Define the levels and their positions exactly as in screenshot
            levels = ['Beginner', 'Intermediate', 'Advanced', 'Expert']
            level_positions = [1, 2, 3, 4]  # Vertical positions

            # Set up the graph background and styling
            ax.set_facecolor('#f8f9fa')
            fig.patch.set_facecolor('#f8f9fa')

            # Create the main diagonal line (the graph line)
            x_points = [5, 6, 7, 8, 9]
            y_points = [1, 2, 3, 4, 5]  # Slightly offset for visual appeal

            # Plot the diagonal graph line
            ax.plot(x_points, y_points, color='#0d6efd', linewidth=3, marker='o',
                    markersize=8, markerfacecolor='white', markeredgecolor='#0d6efd',
                    markeredgewidth=2)

            # Add level labels on the left side (like in screenshot)
            for level, pos in zip(levels, level_positions):
                ax.text(4.7, pos, level, ha='right', va='center',
                       fontweight='bold', fontsize=11, color='#495057')

            # Add "Graph Level" title on the left
            ax.text(4.7, 4.8, 'Graph Level', ha='right', va='center',
                   fontweight='bold', fontsize=12, color='#212529')

            # Add "Graph" label above the line
            ax.text(7, 5.2, 'Graph', ha='center', va='center',
                   fontweight='bold', fontsize=11, color='#212529')

            # Add "Diagonal" label
            ax.text(7, 0.7, 'Diagonal', ha='center', va='center',
                   fontstyle='italic', fontsize=10, color='#6c757d')

            # Add "Diagrams" label (positioned similarly to screenshot)
            ax.text(8.5, 3.5, 'Diagrams', ha='center', va='center',
                   fontstyle='italic', fontsize=10, color='#6c757d', rotation=-45)

            # Add "Intermediate" indicator (the main focus point)
            intermediate_x = 7
            intermediate_y = 3
            ax.plot(intermediate_x, intermediate_y, 's', markersize=12,
                    color='#dc3545', markerfacecolor='#dc3545')

            # Add "Intermediate" text label
            ax.text(7.3, 3, 'Intermediate', ha='left', va='center',
                   fontweight='bold', fontsize=10, color='#dc3545')

            # Add "Acknowledgment" text
            ax.text(5.5, 0.4, 'Acknowledgment', ha='center', va='center',
                   fontsize=9, color='#6c757d')

            # Set axis limits and labels
            ax.set_xlim(4.5, 9.5)
            ax.set_ylim(0.5, 5.5)

            # Remove axes spines and ticks
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.spines['bottom'].set_visible(False)

            # Remove ticks
            ax.set_xticks([])
            ax.set_yticks([])

            # Add grid lines for better readability (subtle)
            ax.grid(True, alpha=0.2, linestyle='-', color='#adb5bd')

            # Add a subtle border
            rect = plt.Rectangle((4.5, 0.5), 5, 5, linewidth=1, edgecolor='#dee2e6',
                               facecolor='none', linestyle='-')
            ax.add_patch(rect)

            plt.tight_layout()
            return fig

        except Exception as e:
            st.error(f"Error creating complexity chart: {e}")
            return None

    # Main paraphrasing logic
    if generate_clicked and input_text.strip():
        if paraphrase_model is None:
            st.error("❌ Paraphrase model not loaded. Please refresh the page.")
        else:
            with st.spinner(f"Generating {complexity_level.lower()} paraphrase using {model_type}..."):
                start_time = time.time()
                # Use local model instead of API
                paraphrased_text = local_paraphrase(
                    paraphrase_model, paraphrase_tokenizer, input_text, complexity_level, paraphrasing_style
                )
                end_time = time.time()

                original_grade = calculate_flesch_kincaid_para(input_text)
                new_grade = calculate_flesch_kincaid_para(paraphrased_text)

                # Add to history
                add_to_paraphrase_history(
                    input_text, paraphrased_text, model_type, complexity_level,
                    paraphrasing_style, original_grade, new_grade
                )

                # Display results
                st.markdown("---")
                st.subheader("📊 Paraphrase Results")

                # Side-by-side comparison
                col1, col2 = st.columns(2)

                with col1:
                    st.markdown("**Original Text**")
                    st.markdown(f"**Grade Level: {original_grade:.1f}**")
                    st.info(input_text)

                with col2:
                    st.markdown("**Paraphrased Text**")
                    st.markdown(f"**Grade Level: {new_grade:.1f}**")
                    st.success(paraphrased_text)

                # Complexity visualization
                st.markdown("---")
                st.subheader("📈 Complexity Visualization")

                complexity_chart = create_complexity_chart(original_grade, new_grade, complexity_level)
                if complexity_chart:
                    st.pyplot(complexity_chart)

                # Grade level interpretation
                st.markdown("---")
                st.subheader("📖 Grade Level Interpretation")

                col1, col2, col3, col4 = st.columns(4)

                with col1:
                    st.metric("Original Grade", f"{original_grade:.1f}")
                with col2:
                    st.metric("New Grade", f"{new_grade:.1f}")
                with col3:
                    change = new_grade - original_grade
                    st.metric("Change", f"{change:+.1f}")
                with col4:
                    if abs(change) <= 1:
                        st.metric("Impact", "Minimal")
                    elif abs(change) <= 3:
                        st.metric("Impact", "Moderate")
                    else:
                        st.metric("Impact", "Significant")

                # Show performance info
                st.info(f"⏱️ Generated in {end_time - start_time:.2f} seconds")

                # Detailed interpretation
                with st.expander("💡 Understanding Grade Levels"):
                    st.markdown("""
                    **Grade Level Interpretation Guide:**

                    - **1-4**: Elementary school level
                    - **5-8**: Middle school level
                    - **9-12**: High school level
                    - **13-16**: College level
                    - **17-20**: Graduate/professional level

                    **Complexity Adjustment:**
                    - **Negative change**: Text simplified
                    - **Positive change**: Text made more complex
                    - **Minimal change**: Complexity maintained
                    """)

    elif generate_clicked and not input_text.strip():
        st.warning("Please enter some text or upload a file to paraphrase.")

    # History section
    st.markdown("---")
    st.subheader("📜 Paraphrasing History")

    if st.session_state.paraphrasing_history:
        history_data = []
        for entry in st.session_state.paraphrasing_history:
            history_data.append({
                'Timestamp': entry['timestamp'],
                'Model': entry['model'],
                'Complexity': entry['complexity'],
                'Style': entry['style'],
                'Original Grade': f"{entry['original_grade']:.1f}",
                'New Grade': f"{entry['new_grade']:.1f}",
                'Change': f"{entry['improvement']:+.1f}"
            })

        history_df = pd.DataFrame(history_data)
        st.dataframe(history_df, use_container_width=True)

        # Clear history button
        if st.button("Clear History", type="secondary"):
            st.session_state.paraphrasing_history = []
            st.rerun()
    else:
        st.info("No paraphrasing history yet. Generate some paraphrases to see them here!")

    # Key Features section
    st.markdown("---")
    st.subheader("🔑 Key Features")

    feature_col1, feature_col2, feature_col3 = st.columns(3)

    with feature_col1:
        st.markdown("""
        **🤖 Advanced NLP Models**
        - FLAN-T5: Instruction-tuned paraphrasing
        - BART: Denoising autoencoder
        - Multi-level complexity adjustment
        """)

    with feature_col2:
        st.markdown("""
        **🎯 Targeted Complexity**
        - Beginner: Simple language
        - Intermediate: Balanced complexity
        - Advanced: Sophisticated vocabulary
        - Expert: Professional/technical
        """)

    with feature_col3:
        st.markdown("""
        **📊 Visual Analytics**
        - Grade level comparison
        - Complexity progression charts
        - Side-by-side text comparison
        """)

# =============================================================================
#  UI SECTION: CHAT INTERFACE (PLACEHOLDER)
# =============================================================================

def chat_interface():
    st.title("💬 Chat with AI")
    st.info("🚧 This feature is under development")
    st.write("""
    Interactive chat interface for:

    - **Text Analysis**: Ask questions about your documents
    - **Writing Assistance**: Get help with writing and editing
    - **Content Generation**: Create new content based on your needs
    - **Learning Support**: Get explanations and insights

    Start a conversation with our AI assistant!
    """)

    # Placeholder for future implementation
    if "chat_messages" not in st.session_state:
        st.session_state.chat_messages = []

    for message in st.session_state.chat_messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])

    if prompt := st.chat_input("Ask me anything..."):
        st.session_state.chat_messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.write(prompt)

        # Simulate AI response
        with st.chat_message("assistant"):
            st.write("This is a simulated response. The chat feature is currently under development.")

# =============================================================================
#  UI SECTION: ADMIN DASHBOARD
# =============================================================================

def admin_dashboard():
    st.title("👑 Admin Dashboard")
    tab1, tab2 = st.tabs(["User Management", "App Analytics (Placeholder)"])
    with tab1:
        st.subheader("User Database")
        users_df = get_all_users()
        st.dataframe(users_df, use_container_width=True)
        col1, col2 = st.columns(2)
        with col1:
            with st.form("add_user_form", clear_on_submit=True):
                st.subheader("➕ Add New User")
                new_email = st.text_input("New User Email")
                new_password = st.text_input("New User Password", type="password")
                new_role = st.selectbox("Assign Role", ["General User", "Admin"])
                if st.form_submit_button("Add User"):
                    if new_email and new_password:
                        message = register_user(new_email, new_password, new_role)
                        st.success(message)
                        st.rerun()
                    else:
                        st.error("Please fill in all fields.")
        with col2:
            with st.form("delete_user_form", clear_on_submit=True):
                st.subheader("⚠️ Delete User")
                user_to_delete = st.selectbox("Select User to Delete", options=users_df['email'].tolist())
                if st.form_submit_button("Delete User", type="primary"):
                    delete_user(user_to_delete)
                    st.warning(f"User '{user_to_delete}' deleted.")
                    st.rerun()
    with tab2:
        st.subheader("Application Analytics")
        st.metric("Total Users", len(get_all_users()))
        st.metric("Daily Active Users (Mock)", "123")
        chart_data = pd.DataFrame(np.random.randn(20, 3), columns=['A', 'B', 'C'])
        st.line_chart(chart_data)

# =============================================================================
#  NAVIGATION FUNCTION
# =============================================================================
def render_navigation():
    """Render the navigation menu in sidebar"""
    payload = decode_token(st.session_state.token)

    if payload and payload['role'].lower() == 'admin':
        # For admin users - only show Settings and Logout, no navigation menu
        st.sidebar.markdown("## 👑 Admin Panel")
        st.sidebar.info("Admins have access to all features through the Admin Dashboard")
    else:
        # For regular users - show full navigation menu
        st.sidebar.markdown("## 📱 Navigation")

        # Navigation options for regular users
        nav_options = {
            "📊 Dashboard & Readability Analysis": "dashboard",
            "📝 Multi-level Summarization": "summarization",
            "🔄 Complexity-based Paraphrasing": "paraphrasing",
            "💬 Chat": "chat"
        }

        # Create navigation buttons
        for option, key in nav_options.items():
            if st.sidebar.button(option, key=f"nav_{key}", use_container_width=True):
                st.session_state.current_page = key

        st.sidebar.markdown("---")

# =============================================================================
#  MAIN AUTHENTICATION ROUTER
# =============================================================================

if 'token' not in st.session_state:
    st.session_state.token = None
if 'theme' not in st.session_state:
    st.session_state.theme = "Light"
if 'current_page' not in st.session_state:
    st.session_state.current_page = "dashboard"
if 'forgot_password_stage' not in st.session_state:
    st.session_state.forgot_password_stage = "request"

# Initialize model histories
if 'summarization_history' not in st.session_state:
    st.session_state.summarization_history = []
if 'paraphrasing_history' not in st.session_state:
    st.session_state.paraphrasing_history = []

payload = decode_token(st.session_state.token)

if payload is None:
    st.session_state.token = None
    # --- UPDATED: Use columns to center the login form ---
    st.markdown("<style>div[data-testid='stHorizontalBlock'] { text-align: center; }</style>", unsafe_allow_html=True)
    _ , main_content, _ = st.columns([1, 1.5, 1])
    with main_content:
        st.markdown("<h1>➡ User Authentication</h1>", unsafe_allow_html=True)
        login_tab, register_tab, forgot_tab = st.tabs(["Login", "Register", "Forgot Password"])
        with login_tab:
            with st.form("login_form"):
                email = st.text_input("Email", placeholder="Enter your Email")
                password = st.text_input("Password", type="password", placeholder="*****")
                if st.form_submit_button("Sign In"):
                    token = authenticate_user(email, password)
                    if token:
                        st.session_state.token = token
                        st.rerun()
                    else:
                        st.error("Invalid email or password.")
        # --- UPDATED: Added full registration logic ---
        with register_tab:
            with st.form("register_form"):
                new_email = st.text_input("Email", key="reg_email")
                new_password = st.text_input("New Password", type="password", key="reg_pass")
                confirm_password = st.text_input("Confirm Password", type="password", key="reg_confirm")
                role = st.selectbox("Role", ["General User", "Admin"])
                if st.form_submit_button("Register"):
                    if new_password == confirm_password:
                        message = register_user(new_email, new_password, role)
                        if "successfully" in message:
                            st.success(message)
                        else:
                            st.warning(message)
                    else:
                        st.error("Passwords do not match.")
        with forgot_tab:
            forgot_password_flow()
else:
    role = payload['role']
    with st.sidebar:
        st.success(f"Logged in as: {payload['sub']}")
        st.write(f"Your role is: **{role}**")

        # Render navigation menu
        render_navigation()

        if st.button("Logout"):
            st.session_state.token = None
            st.rerun()
        st.markdown("---")
        with st.expander("⚙️ Settings"):
            st.session_state.theme = st.radio("Theme", ["Light", "Dark"])

    if st.session_state.theme == "Dark":
        st.markdown(dark_theme_css, unsafe_allow_html=True)

    # Page routing based on navigation
    if role.lower() == "admin":
        admin_dashboard()
    else:
        if st.session_state.current_page == "dashboard":
            readability_dashboard()
        elif st.session_state.current_page == "summarization":
            multi_level_summarization()
        elif st.session_state.current_page == "paraphrasing":
            complexity_paraphrasing()
        elif st.session_state.current_page == "chat":
            chat_interface()
        else:
            readability_dashboard()  # Default page

Writing app1.py


# SECTION 3: Running StreamLit

In [3]:
# Replace with your actual ngrok authtoken
!ngrok config add-authtoken 34Nkqo7OM708MU0EvgUjzlPSEt2_84PRCVh5FgpuKVEQiYgLX
ngrok.kill()

public_url = ngrok.connect(8502)
print(f"🌍 Public URL: {public_url}")
!streamlit run app1.py --server.port 8502 --server.headless true

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Public URL: NgrokTunnel: "https://subcultural-kimberli-unretrogressively.ngrok-free.dev" -> "http://localhost:8502"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://136.117.156.218:8502

2025-10-28 12:36:28.817930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761654988.882608     808 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761654988.898165     808 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761654988.938224     808 computation_placer.cc:177] computa